# BW Project Title

## Team Members
- Name #1
- ...

## Project Description
Insert a short 1-2 sentence description of what you plan to build.

In [2]:
# Import required packages
import numpy as np
import tensorflow as tf

print(f'Numpy Version: {np.__version__}')
print(f'Tensorflow Version: {tf.__version__}')

Numpy Version: 1.19.2
Tensorflow Version: 2.4.0


## Load Data

In [1]:
import pandas as pd
import numpy as np
import zipfile

ModuleNotFoundError: No module named 'pandas'

## Data Exploration

In [ ]:
# Descriptive statistics for key fields
# Add visualizations (where applicable)

## Baseline Model (Benchmark)

In [ ]:
# Simple baseline model

## Modeling Approach #1

## Modeling Approach #2

## Hyper-parameter Tuning for Best Model

## Save Final Model

In [ ]:
import joblib

joblib.dump(model, '../models/model_name.joblib')